In [1]:
#!pip install pyspark

In [11]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import *
import numpy as np

In [2]:
#connector 설정

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('sql') \
    .set("spark.driver.extraClassPath","jdbc/mysql-connector-java-5.1.44.jar")

sc = SparkContext(conf=conf)

spark = SparkSession(sc)

21/09/09 09:26:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/09 09:26:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#db접속정보 설정
                        #[ip주소:port]   #[db명]                 #[유저명]          #[비밀번호]
DB_URL = 'jdbc:mysql://                /       ?useSSL=false&user=        &password=           '


In [7]:
#데이터파일(csv)을 DataFrame 으로 읽어들인다.
# inferSchema : 데이터를 기반으로 타입을 유추(기본값 : False)
# header : 첫번째 행을 컬럼 값으로 사용

df = spark.read.csv("data/전국금연구역표준데이터.csv", encoding = 'cp949',inferSchema = True, header = True)
df.show(2)

+----------+----------------+--------+--------+------------+------------------+------------+----------+----------------+------------------------------+--------------+---------------+-------------+--------------+--------------+------------+---------------+
|금연구역명|금연구역범위상세|  시도명|시군구명|금연구역구분|금연구역지정근거명|금연구역면적|위반과태료|위반신고전화번호|              소재지도로명주소|소재지지번주소|     관리기관명|         위도|          경도|데이터기준일자|제공기관코드|     제공기관명|
+----------+----------------+--------+--------+------------+------------------+------------+----------+----------------+------------------------------+--------------+---------------+-------------+--------------+--------------+------------+---------------+
|    어미향|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|    100000|    061-277-4000|전라남도 목포시 동명로 19, ...|          null|전라남도 목포시|  34.78903148|   126.3955929|    2020-10-08|     4800000|전라남도 목포시|
|  지맘대로|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|    100000|    061-277-4000|전라남도 목포시 동명로 23, ...|          

In [8]:
#컬럼정보 확인
df.printSchema

<bound method DataFrame.printSchema of DataFrame[금연구역명: string, 금연구역범위상세: string, 시도명: string, 시군구명: string, 금연구역구분: string, 금연구역지정근거명: string, 금연구역면적: double, 위반과태료: int, 위반신고전화번호: string, 소재지도로명주소: string, 소재지지번주소: string, 관리기관명: string, 위도: double, 경도: double, 데이터기준일자: string, 제공기관코드: string, 제공기관명: string]>

In [9]:
# DataFrame 타입인 것을 확인
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [29]:
# 몇몇 컬럼만 선택한 결과는 새로운 DataFrame 으로 반완됨
print(type(df.select('금연구역명', '금연구역범위상세', '시도명', '시군구명','금연구역구분','금연구역지정근거명','금연구역면적')))
# 데이터 조회
df.select('금연구역명', '금연구역범위상세', '시도명', '시군구명','금연구역구분','금연구역지정근거명','금연구역면적').show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
+----------+----------------+--------+--------+------------+------------------+------------+
|금연구역명|금연구역범위상세|  시도명|시군구명|금연구역구분|금연구역지정근거명|금연구역면적|
+----------+----------------+--------+--------+------------+------------------+------------+
|    어미향|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|
|  지맘대로|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|
|      미담|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|
|    지니빠|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|
|1004의쉼터|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|
+----------+----------------+--------+--------+------------+------------------+------------+
only showing top 5 rows



In [30]:
# 정보분리
df_new = df.select('금연구역명', '금연구역범위상세', '시도명', '시군구명','금연구역구분','금연구역지정근거명','금연구역면적')
df_new.show(5)

+----------+----------------+--------+--------+------------+------------------+------------+
|금연구역명|금연구역범위상세|  시도명|시군구명|금연구역구분|금연구역지정근거명|금연구역면적|
+----------+----------------+--------+--------+------------+------------------+------------+
|    어미향|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|
|  지맘대로|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|
|      미담|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|
|    지니빠|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|
|1004의쉼터|    건물 내 금연|전라남도|  목포시|  일반음식점|    국민건강증진법|        null|
+----------+----------------+--------+--------+------------+------------------+------------+
only showing top 5 rows



In [31]:
#DB에 저장
df_new \
    .write \
    .format('jdbc') \
    .mode('append') \
    .option("url", DB_URL) \
    .option("dbtable", "no_smomking") \
    .save()